# Kustomize (manage YAML at scale)

Kubernetes starts with “raw YAML” manifests. That works early, but teams quickly hit:
- **Copy/paste drift** across environments
- Many near-identical files (dev/stage/prod)
- Hard-to-review changes (what differs between envs?)

**Kustomize** solves this by letting you define a **base** set of resources and then apply **overlays** (patches) per environment—without introducing a templating language.


## Goals
- Understand `kustomization.yaml` and the **base/overlay** pattern.
- See how overlays reduce copy/paste and prevent env drift.
- Learn common Kustomize features: `resources`, `patches`, `images`, generators, `namespace`.
- Know when **Helm** might be a better fit.


## What it is
- **Kustomize** is a manifest customization tool.
- You keep valid Kubernetes YAML.
- A `kustomization.yaml` file declares how to compose/transform resources.

Kustomize is built into `kubectl`:
```bash
kubectl apply -k ./overlays/prod
kubectl kustomize ./overlays/prod | less
```


## Base vs overlays (the pattern)
Typical structure:

```text
app/
  base/
    deployment.yaml
    service.yaml
    kustomization.yaml
  overlays/
    dev/
      kustomization.yaml
    prod/
      kustomization.yaml
      patch-replicas.yaml
```

- **base**: shared, environment-agnostic defaults.
- **overlay**: environment-specific changes (replicas, images, namespaces, ingress hosts, resources).


## Pseudocode: how Kustomize works
```text
load resources from base
apply overlay transforms:
  - set namespace
  - change image tags
  - apply patches
  - generate configmaps/secrets (optional)
output final YAML
```


## Minimal example (files in this repo)
This folder includes a runnable skeleton example:
- `kubernetes/kustomize/_example/base/kustomization.yaml`
- `kubernetes/kustomize/_example/overlays/dev/kustomization.yaml`
- `kubernetes/kustomize/_example/overlays/prod/kustomization.yaml`

Build the prod manifests:
```bash
kubectl kustomize kubernetes/kustomize/_example/overlays/prod
```

Apply them:
```bash
kubectl apply -k kubernetes/kustomize/_example/overlays/prod
```


## Common Kustomize features you’ll actually use
- **`namespace:`** in overlays to prevent cross-env collisions.
- **`images:`** to swap tags without editing Deployments.
- **`patches:`** to change replicas/resources/ingress hosts.
- **Generators** (`configMapGenerator`) for simple config injection.
- **Common labels/annotations** to tag everything with `env`, `team`, `app`.


## Kustomize vs Helm (rule of thumb)
Use **Kustomize** when:
- You want to stay close to raw YAML.
- You mostly need overlays/patches, image tags, namespaces.

Use **Helm** when:
- You need a **packaged** app with versions and dependencies.
- You want templating for many variations.
- You’re consuming third-party charts (Prometheus, ingress controllers, etc.).

Many teams do both: Helm for installing shared infrastructure, Kustomize for app manifests.


## Pitfalls
- Generators can change resource names (hash suffixes) unless controlled; know how that affects references.
- Keep base clean; avoid environment-specific values leaking into base.
- Patch carefully; prefer targeting by `kind` + `name`.

## References
- Kustomize docs: https://kubectl.docs.kubernetes.io/references/kustomize/
- kubectl apply -k: https://kubernetes.io/docs/tasks/manage-kubernetes-objects/kustomization/
